In [1]:
from abc import ABC, abstractmethod
from datetime import datetime, timedelta, date
import os

import housekeeper

class DataManager(ABC):
    
    def __init__(self):
        
        self.__myHousekeeper = housekeeper.instance_class()
        self.__config_filename = "tickers_config.json"
        self.__dir_list = ['Data', 'Tickers', 'Dummy1']
        self.__upper_stages = 0
        self.__tickers_config_list = []
        self.__tickers_list = []
        self.__active_tickers_list = []
        self.__selected_tickers_list = []
        self.__timestamp = ''
        self.__markets = []
    

    
    def get_config_filename(self):
        return self.__config_filename
    
    def set_config_filename(self, config_filename):
        self.__config_filename = config_filename
        
    def get_dir_list(self):
        return self.__dir_list
    
    def set_dir_list(self, dir_list):
        self.__dir_list = dir_list
    
    def get_upper_stages(self):
        return self.__upper_stages
    
    def set_upper_stages(self, upper_stages):
        self.__upper_stages = dir_list
    
    def get_tickers_config(self):
        return self.__tickers_config_list
    
    def set_tickers_config(self, tickers_config_list):
        self.__tickers_config_list = tickers_config_list
    
    def get_tickers(self):
        return self.__tickers_list
    
    def set_tickers(self, tickers_list):
        self.__tickers_list = tickers_list
        
    def get_active_tickers(self):
        return self.__active_tickers_list
    
    def set_active_tickers(self, active_tickers_list):
        self.__active_tickers_list = active_tickers_list
        
    def get_selected_tickers(self):
        return self.__selected_tickers_list
    
    def set_selected_tickers(self, selected_tickers_list):
        self.__selected_tickers_list = selected_tickers_list
    
    def get_timestamp(self):
        return self.__timestamp
    
    def set_timestamp(self, timestamp):
        self.__timestamp = timestamp
    
    def get_markets(self):
        return self.__markets
    
    def set_markets(self, markets):
        self.__markets = markets
    
    def load_tickers_config(self):
        data = self.__myHousekeeper.load_json_to_list(self.__dir_list, self.__config_filename)
        self.set_tickers_config(data)
        
    def save_tickers_config(self):
        #No invocar a esta función sin previamente haber cargado tickers_config. O se sobreescribe tickers_config
        tickers_config = self.get_tickers_config()
        self.__myHousekeeper.list_dict_to_json(self.get_dir_list(), 
                                               self.get_upper_stages(), 
                                               self.get_config_filename(), 
                                               self.get_tickers_config())
    
    def initialize_metadata(self):
        self.load_tickers_config()
        data = self.get_tickers_config()
        self.set_timestamp(data['metadata'][0]['timestamp'])
        self.set_tickers(data['data'])
        
    def initialize_config_tickers(self):
        # Get markets, get active_tickers
        markets = []
        active_tickers_ = []
        self.initialize_metadata()
        data = self.get_tickers()
        for d in data:
            markets.append(d['market'])
            if d['active_type']=='stock' and d['active_flag']:
                active_tickers_.append(d)
            elif d['active_type']=='ETF':
                active_tickers_.append(d)
        self.set_active_tickers(active_tickers_)
        self.set_markets(list(set(markets)))
    
    def api_selected_tickers(self):
        # Se despliegan los tickers activos en la UI para que el usuario elija qué tickers quiere actualizar el data.
        ticker_list = self.get_tickers()
        self.set_selected_tickers(ticker_list[0:2])
        #return self.get_active_tickers() #TODO
    
    def update_timeseries_download_date(self):
        # Se actualiza el tickers_config.json incluyendo la fecha en la que se han descargado datos asociados a un ticker.
        self.api_selected_tickers()
        selected_ticker_key = [t['tickerKey'] for t in self.get_selected_tickers()]
        config_ticker_list = self.get_tickers_config()
        today = date.today()
        [t.update({'data_update':today.strftime("%d-%m-%Y")}) for t in config_ticker_list['data'] if t['tickerKey'] in selected_ticker_key]
        self.set_tickers_config(config_ticker_list)
        self.save_tickers_config()
         
    def save_ticker_data(self, file_name, data):
        self.__myHousekeeper.df_to_csv(self.__dir_list,
                                       self.__upper_stages, file_name, data)
        
    

In [2]:
import yfinance as yf
import pandas as pd

class DataManager_YahooFinance(DataManager):
    
    def __init__(self):
        super().__init__()
        
    def download_ticker_data(self, ticker, ticker_key):
        print('Downloading historic data of: ' + ticker)
        data_csv = yf.download(ticker)
        data_csv.insert(loc=0, column='Date', value=pd.to_datetime(data_csv.index, errors='coerce'))
        data_csv['Date'] = [time.date() for time in data_csv['Date']]
        data_csv.reset_index(drop=True, inplace=True)
        self.save_ticker_data(ticker_key,data_csv )
        return data_csv
    
    def download_market_data(self, market):
        print('Download market ticker')
        tickers = self.get_active_tickers()
        [DM_YF.download_ticker_data(t['feeds']['ticker'], t['tickerKey']) for t in tickers if t['market'] in markets]
    
    def download_all_markets(self):
        print('Download ALL MARKETS')
        self.download_market_data(self.get_markets())
    
    def download_selected_tickers(self):
        # Se almacenan los tickers que van a se actualizados y se guarda la fecha de actualización en el ticker_config. 
        # 1.- Almacenar selected_Tickers from user intertest and a default option.
        #selected_tickers_list = self.api_active_tickers()
        self.api_selected_tickers()
        #2.- Descargar los selected_tickers
        [self.download_ticker_data(t['feeds']['ticker'], t['tickerKey']) for t in self.get_selected_tickers()]
        # 3.- Actualizar fechas en tickers_config
        self.update_timeseries_download_date()
    

        

In [3]:
DM_YF = DataManager_YahooFinance()
DM_YF.load_tickers_config() # Cargar archivo "tickers_config.json"
data = DM_YF.get_tickers_config() # Obtener variable __tickers_config_list
#DM_YF.initialize_metadata()
DM_YF.initialize_config_tickers() # initialize_metadata(timestamp and set tickers_list) and set __active_tickers_list 
DM_YF.get_timestamp(), DM_YF.get_markets()

nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/tickers_config.json
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/tickers_config.json


('21/03/2021 13:02:45', ['SP500', 'CAC40', 'IBEX35', 'DAX30'])

In [6]:
DM_YF.download_selected_tickers()

[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ANA.MC.TT.csv
[*********************100%***********************]  1 of 1 completed
nested_dir_path:  /Data/Tickers/Dummy1
./Data/Tickers/Dummy1/ACX.MC.TT.csv
Root dir:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project
parent_dir_path:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project
dir_list:  ['Data', 'Tickers', 'Dummy1']
nested_dir_path:  /Data/Tickers/Dummy1
file path:  C:\Users\alvaro\Repos\Python_knowledge\Appdatabase project/Data/Tickers/Dummy1/tickers_config.json


In [10]:
a = DM_YF.get_active_tickers()
a[0]['data_update'], a[2]['data_update']

('25-03-2021', '-')

In [26]:
#data_csv = yf.download('ANA.MC')
last_date = a[0]['data_update']
last_date = pd.to_datetime(last_date, errors='coerce')
last_date.date()
start_date = last_date + timedelta(days=1)
start_date_1 = last_date + timedelta(days=-2)
last_date, start_date, start_date_1

(Timestamp('2021-03-25 00:00:00'),
 Timestamp('2021-03-26 00:00:00'),
 Timestamp('2021-03-23 00:00:00'))

In [32]:
data_csv = yf.download('ANA.MC', start=start_date_1, end=datetime.now())
#data_csv = yf.download('ANA.MC')
data_csv.head(-1)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-03-23,136.399994,138.899994,134.600006,138.000000,138.000000,118561
2021-03-24,136.800003,138.300003,136.000000,138.199997,138.199997,176248


In [14]:
def f1(a = 1):
    print(a)

In [19]:
f1(424)

424


In [ ]:
tickers[0]

In [ ]:
tickers[0]['feeds']['ticker']

In [ ]:
data_ticker = DM_YF.download_ticker_data(tickers[0]['feeds']['ticker'], tickers[0]['tickerKey'])
#data_ticker

In [ ]:
data_ticker.head(), type(data_ticker)

In [ ]:
#DM_YF.download_market_data('IBEX35')

In [ ]:
markets = DM_YF.get_markets()
markets

In [ ]:
DM_YF.download_all_markets()

In [ ]:
def Get_Last_Data(ticker, market, backup):
    # Check if the market is in the database. In case that not exist, WARNING and return empty csv.
    markets = os.listdir('./Data/')
    if market not in markets:
        print("There is no market in database, please add new market and fill with data")
        data_csv = []
        return data_csv

    # If market exists between the markets loaded, then go to market folder and load shares list
    # Get shares of a market from the folder that it is stored
    shares = os.listdir('./Data/' + market)
    shares = [s.replace('.csv', '') for s in shares]

    # Function to download full historic data, used for backup and in firts time download
    def dowload_historic_data(ticker):
        print('Downloading historic data of: ' + ticker)
        data_csv = yf.download(ticker)
        data_csv.insert(loc=0, column='Date', value=pd.to_datetime(data_csv.index, errors='coerce'))
        data_csv['Date'] = [time.date() for time in data_csv['Date']]
        data_csv.reset_index(drop=True, inplace=True)
        return data_csv

    if ticker in shares:  # IN CASE THAT THE SHARE EXISTS PREVIOUSLY. If share in LISTA_ACCIONES
        if backup:  # BACK UP CASE. Download data from the historic in YahooFinance
            data_csv = dowload_historic_data(ticker)
        else:  # NO BACK UP CASE. Download data from the last date updated
            # Load de data
            print("-------------------------------------------")
            print(ticker + " exists in Database. " + "Opening " + ticker)
            data_csv = pd.read_csv("./Data/" + market + "/" + ticker + ".csv")
            # Read and get the date of the last day
            last_date = data_csv["Date"].iloc[-1]
            last_date = pd.to_datetime(last_date, errors='coerce')
            last_date.date()
            # If last date is today, then return the file with no modifications
            if last_date >= date.today():
                print("File is already updated. No modifications.")
                # print ("-------------------------------------------")
                return data_csv
            # print("Updating " + ticker +' from '+ last_date +' until '+date.today()+'(today)' )
            print('Updating ' + ticker + ' until today')
            start_date = last_date + timedelta(days=1)
            # Download data from the selected date
            data = yf.download(ticker, start=start_date, end=datetime.now())
            data.insert(loc=0, column='Date', value=pd.to_datetime(data.index, errors='coerce'))
            data['Date'] = [time.date() for time in data['Date']]
            data.reset_index(drop=True, inplace=True)
            data_csv = data_csv.append(data, ignore_index=True)
    else:  # IN CASE THAT THE SHARE DOESNT EXISTS PREVIOUSLY. Download historic data.
        print(ticker + " is not found in database, adding ----")
        data_csv = dowload_historic_data(ticker)
    return data_csv

### Development layer 

In [ ]:
def download_market_data(self, market):
    print('Download market ticker')
    tickers = self.get_active_tickers()
    [DM_YF.download_ticker_data(t['feeds']['ticker'], t['tickerKey']) for t in tickers if t['market'] in markets]

In [ ]:
tickers = DM_YF.get_active_tickers() # Get active tickers
#tickers

In [ ]:
markets = ['IBEX35']
[print(t) for t in tickers if t['market'] in markets]

DM_YF.download_ticker_data(tickers[0]['feeds']['ticker'], tickers[0]['tickerKey'])

In [ ]:
[DM_YF.download_ticker_data(t['feeds']['ticker'], t['tickerKey']) for t in tickers if t['market'] in markets]

In [ ]:
import housekeeper

In [ ]:
dir_list = ['Data']
file_name = 'tickers_config.json'

In [ ]:
myHousekeeper = housekeeper.instance_class() 
data = myHousekeeper.load_json_to_list(dir_list, file_name)
#data

In [ ]:
markets = []
active_tickers = []
for d in data:
    markets.append(d['market'])
    if d['active_type']=='stock' and d['active_flag']:
        active_tickers.append(d)
    elif d['active_type']=='ETF':
        active_tickers.append(d)
markets = set(markets)    

In [ ]:
markets, active_tickers

In [ ]:
data[0:2]['market']

In [ ]:
data[0:2]

In [ ]:
keyValList = ['a']
data[0]['market'] in keyValList

In [ ]:
#data

In [ ]:
if data[0]['active_flag']:
    print('hola')

In [ ]:
markets

In [ ]:
'''
>>> exampleSet = [{'type':'type1'},{'type':'type2'},{'type':'type2'}, {'type':'type3'}]
>>> keyValList = ['type2','type3']
>>> expectedResult = [d for d in exampleSet if d['type'] in keyValList]
>>> expectedResult
[{'type': 'type2'}, {'type': 'type2'}, {'type': 'type3'}]
'''

In [ ]:
'''
>>> list(filter(lambda d: d['type'] in keyValList, exampleSet))
[{'type': 'type2'}, {'type': 'type2'}, {'type': 'type3'}]
'''

https://blog.finxter.com/how-to-filter-a-list-of-dictionaries-in-python/#Where_to_Go_From_Here

In [ ]:
data = DM_YF.get_tickers_config()

In [ ]:
DM_YF.load_tickers_config()

In [ ]:
data[1]

In [ ]:
list = ['a', 'b', 'c']

In [ ]:
list.pop(0)
list

In [ ]:
a = list.pop(0)
a